In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# %cd ./drive/My\ Drive/Model


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Model


In [2]:
# !ls

' (1).ipynb_checkpoints'	      intraday			 model.h5
 config.json			      merged.csv		 model.ipynb
 core				      merged_test.csv		 README.md
'Hyperparameter optimization.ipynb'  'Model Documentation.pdf'	 saved_models


In [1]:
import scipy.signal as sci
import scipy.stats as scp
import pandas as pd
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from math import sqrt
import math
import os
import json
import time
import seaborn as sns
from core.data_processor import DataLoader
from core.model import Model
import datetime


Using TensorFlow backend.


#  **Hyper parameters to tune**


*   No: of epocs
*   Batch size
*   Architecture like number of layers, neurons, dropout rate



In [2]:
configs = json.load(open('config.json', 'r'))

In [10]:
df_std = pd.read_csv('merged.csv')
df_std.head()

,Unnamed: 0,datetime_x,midChangeGroup,futMid,mid,arrival_rate,price,size,imbalance,Trade,spread
0,0,2018-02-01 08:00:18.248,7,4839.5,1.964891,2.148942,0.0,0.0,-0.876897,0,3.477824
1,1,2018-02-01 08:00:18.248,8,4839.5,1.978741,-0.429102,0.0,0.0,-1.618022,0,1.823382
2,2,2018-02-01 08:00:18.248,8,4839.5,1.978741,0.069116,0.0,0.0,-1.927718,0,1.823382
3,3,2018-02-01 08:00:18.248,8,4839.5,1.978741,0.031515,0.0,0.0,-2.089693,0,1.823382
4,4,2018-02-01 08:00:18.249,8,4839.5,1.978741,-0.149442,0.0,0.0,-2.223445,0,1.823382


In [4]:
def model_run():
 
    data = DataLoader(
        configs['data']['filename'],
        configs['data']['train_test_split'],
        configs['data']['columns']
    )

    model = Model()
    model.build_model(configs)
    x, y = data.get_train_data(
        seq_len=configs['data']['sequence_length'],
        normalise=configs['data']['normalise']
    )

    '''
        # in-memory training
        model.train(
            x,
            y,
            epochs = configs['training']['epochs'],
            batch_size = configs['training']['batch_size'],
            save_dir = configs['model']['save_dir']
        )
    '''
    x_test, y_test = data.get_test_data(
        seq_len=configs['data']['sequence_length'],
        normalise=configs['data']['normalise']
    )
    
    # out-of memory generative training
    steps_per_epoch = math.ceil((data.len_train - configs['data']['sequence_length']) / configs['training']['batch_size'])
    model.train_generator(
        data_gen=data.generate_train_batch(
            seq_len=configs['data']['sequence_length'],
            batch_size=configs['training']['batch_size'],
            normalise=configs['data']['normalise']
        ),
        epochs=configs['training']['epochs'],
        batch_size=configs['training']['batch_size'],
        steps_per_epoch=steps_per_epoch,
        save_dir=configs['model']['save_dir']
    )
    
    return model




In [5]:
model = model_run()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 49, 20)            2240      
_________________________________________________________________
dropout_1 (Dropout)          (None, 49, 20)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 49, 30)            6120      
_________________________________________________________________
lstm_3 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 15,711
Trainable params: 15,711
Non-trainable params: 0
_________________________________________________________________
[Model

In [11]:
#Loading saved model
from keras.models import load_model

# returns a compiled model
# identical to the previous one
#To run if not trained the model

model = Model()
model.build_model(configs)
model.load_model('./saved_models/12012019-110735-e10.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 49, 20)            2240      
_________________________________________________________________
dropout_3 (Dropout)          (None, 49, 20)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 49, 30)            6120      
_________________________________________________________________
lstm_6 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dropout_4 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 15,711
Trainable params: 15,711
Non-trainable params: 0
_________________________________________________________________
[Model

In [12]:
#Comparisons of errors 
def error_cal(pred,y):
    rms = sqrt(mean_squared_error(pred,y))
    predCount = len(pred)
    print('RMS = %.4f. #Predictions = %s' % (rms, predCount))
    return rms

data_train = DataLoader(configs['data']['filename'],0.8,configs['data']['columns'])
data_test = DataLoader(configs['data']['test_filename'],0.1,configs['data']['columns'])

#Training error
x_train, y_train = data_train.get_train_data(seq_len=configs['data']['sequence_length'],normalise=configs['data']['normalise'])
pred_train = model.predict_point_by_point(x_train[10000:20000,:,:])
error_train = error_cal(pred_train,y_train[10000:20000])

#Cross-validation error 
x_cv, y_cv = data_train.get_test_data(seq_len=configs['data']['sequence_length'],normalise=configs['data']['normalise'])
pred_cv = model.predict_point_by_point(x_cv[:10000,:,:])
error_train = error_cal(pred_cv,y_cv[:10000])

#Test error
x_test, y_test = data_test.get_test_data(seq_len=configs['data']['sequence_length'],normalise=configs['data']['normalise'])
pred_test = model.predict_point_by_point(x_test[:10000,:,:])
error_test = error_cal(pred_test,y_test[:10000])



[Model] Predicting Point-by-Point...
RMS = 0.1228. #Predictions = 10000
[Model] Predicting Point-by-Point...
RMS = 0.1119. #Predictions = 10000
[Model] Predicting Point-by-Point...
RMS = 0.4325. #Predictions = 10000


(10000, 49, 7)

In [6]:
data = DataLoader(
        configs['data']['filename'],
        configs['data']['train_test_split'],
        configs['data']['columns']
    )

In [0]:
x_test, y_test = data.get_test_data(
        seq_len=configs['data']['sequence_length'],
        normalise=configs['data']['normalise']
    )

In [11]:
y_test.shape

(334501, 1)